In [1]:
import numpy as np

import matplotlib.pyplot as plt

---
# Task1

Запрограммировать формулу точной цены барьерного up-and-out call и сравнить с точным решением европейского call при барьере $B\to\infty$, $B\to K$, $B=2K$, $B=10K$, где К - страйк.

$v(t, x) = x[N(\delta_{+}(\tau, \frac{x}{K})) - N(\delta_{+}(\tau, \frac{x}{B}))] - e^{-r\tau}K[N(\delta_{-}(\tau, \frac{x}{K})) - N(\delta_{-}(\tau, \frac{x}{B}))] - B(\frac{x}{B})^{-\frac{2r}{\sigma^2}}[N(\delta_{+}(\tau, \frac{B^2}{Kx})) - N(\delta_{+}(\tau, \frac{B}{x}))]+e^{-r\tau}K(\frac{x}{B})^{-\frac{2r}{\sigma^2}+1}[N(\delta_{-}(\tau, \frac{B^2}{Kx})) - N(\delta_{-}(\tau, \frac{B}{x}))]$ <br>

$0\leq t < T$, $0< x \leq B$ <br>

$N(x)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^xe^{-\frac{y^2}{2}}dy$ - функция нормального распределения <br>

$\delta_{\pm} = \frac{1}{\sigma\sqrt{\tau}}[\log{S} + (r \pm \frac{1}{2}\sigma^2)\tau]$

In [ ]:
T = 1
B = 100
K = 78
S0 = 50

STEPS = 100

_times = np.linspace()

---
# Task2

Найти цену барьерного <b>up-and-out call</b> методом <b>Монте Карло</b> и сравнить с точной ценой. Задача засчитывается, если относительная погрешность будет не более <b>$10^{-3}$</b> (среднее на неескольких запусках.)

---
# Task3

Решить численно уравнение Блэка-Шоулза для барьерного <b>up-and-out call</b> и сравнить с точной ценой. Задача засчитывается, если относительная погрешность будет не более 0.01 для почти всех узов сетки.

---
# Task4

Запрограммировать формулу цены <b>lookback</b> опциона с переменным страйком после понижения размерности. Нарисовать ее график по $z=\frac{x}{y}$

$u(t,z)=(1 + \frac{\sigma^2}{2r})zN(\delta_{+}(\tau, z)) + e^{-r\tau}N(-\delta_{-}(\tau, z)) - \frac{\sigma^2}{2r}e^{-r\tau}z^{1-\frac{2r}{\sigma^2}}N(-\delta_{-}(\tau, z^{-1})) - z$, $0\leq t < T,\ 0 < z \leq 1$

---
# Task 5

Решиль аналогичную задачу 2.2 для <b>lockback</b> опциона с переменным страйком после понижения размерности.

--- 
# Task6

Решиль аналогичную задачу 2.3 для <b>lockback</b> опциона с переменным страйком после понижения размерности.